In [4]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io

In [13]:
path = ("Hvul_TE.fasta")
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display
print("Corpus length - Cantidad de caracteres:", len(text))

chars = sorted(list(set(text)))
print('Caracteres = ',chars)
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
print('char_indices = ', char_indices)
print('indices_char = ', indices_char)
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 300
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    

Corpus length - Cantidad de caracteres: 632969
Caracteres =  [' ', 'a', 'c', 'g', 'n', 't']
Total chars: 6
char_indices =  {' ': 0, 'a': 1, 'c': 2, 'g': 3, 'n': 4, 't': 5}
indices_char =  {0: ' ', 1: 'a', 2: 'c', 3: 'g', 4: 'n', 5: 't'}
Number of sequences: 210890


In [14]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [15]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [20]:
epochs = 6
batch_size = 100

#import sys
#sys.stdout = open("LSTM_output.txt", "w")
for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [1.0, 1.2]:
        
        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print(">seed {}, Diversity: {}". format(sentence, diversity))

        for i in range(500):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print(generated)
        print()

2109/2109 [==============================] - 762s 361ms/step - loss: 1.3637

Generating text after epoch: 0
>seed ccatagcttgaaatgctatctggattatccttgggcgaattattgcataatatcagttaacatgtttgcataatatcagttaacataatagccggatctaactatatatttatggctgcagacacaggaggaagccatgggaaaagtttgtcgactcagggaacatatttcgctctaaatttatgtacatgtacatgtgcctgttgtttttgtgtcagtatacgttgtttttgattttgctctctgattctttccgctgtgcaaattttgtgaacctgactttttttgaatcgaagtaaa, Diversity: 1.0
attacctcgatatttttggtatgtttgcaagacatgtgttgcctttaatggacatacagaaacgaatattaggtcagccggatgaggagtagagagaaatggaggaacaccgggtttgagtgtgaagtgcgtgcagtcgggttgttcaggatgttgttggatgggacgctgggttagttcagctactacctggatctaggagattctgtgcggtggtggctcttctcgttgttcgtctctcaacgggacggatgtcccatgtccgtgtagaagaagtatggtggccggtgggcgcctccgaggccgaccacggtctggacaacagccgatttttggcagcaccctgggcattccgcttagtagcgccaccattgcctcatacgggggcgcttgaggagtggatcacaagccaggctggggactgattgtacggggcttacgctcgaggaaaagagttagcagctgctggcgctcaacgctttgatccgtgcgtcccggga

>seed ccatagcttgaaatgctatctggattatccttgggcgaattattgcataatatcagttaac

ValueError: pvals < 0, pvals > 1 or pvals contains NaNs